In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, UpSampling1D, BatchNormalization, LeakyReLU, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# Load dataset
mitbih_df = pd.read_csv('../datasets/mitbih_100_signals.csv')
nstdb_em_df = pd.read_csv('../datasets/nstdb_em_signal.csv')
nstdb_ma_df = pd.read_csv('../datasets/nstdb_ma_signal.csv')
nstdb_bw_df = pd.read_csv('../datasets/nstdb_bw_signal.csv')

## Preprocessing

In [46]:
# Normalisasi ke [-1, 1]
def normalize(signal):
    min_val = np.min(signal)
    max_val = np.max(signal)
    range_val = max_val - min_val
    if range_val > 0:
        return 2 * (signal - min_val) / range_val - 1
    else:
        return signal

# Terapkan normalisasi [-1, 1]
mitbih_df['signal_norm'] = mitbih_df['V5'].apply(normalize)
nstdb_bw_df['signal_norm'] = nstdb_bw_df['noise2'].apply(normalize)
nstdb_ma_df['signal_norm'] = nstdb_ma_df['noise2'].apply(normalize)
nstdb_em_df['signal_norm'] = nstdb_em_df['noise2'].apply(normalize)

In [47]:
def segment_signal(signal, window_size):
    segments = []
    for i in range(0, len(signal) - window_size, window_size):
        segments.append(signal[i:i + window_size])
    return np.array(segments).reshape(-1, window_size, 1)

window_size = 256
min_len = min(len(mitbih_df), len(nstdb_bw_df), len(nstdb_ma_df), len(nstdb_em_df))

clean_signals = mitbih_df['signal_norm'].values[:min_len]
bw = nstdb_bw_df['signal_norm'].values[:min_len]
ma = nstdb_ma_df['signal_norm'].values[:min_len]
em = nstdb_em_df['signal_norm'].values[:min_len]

# Gabungkan noise
combined_noise = (bw + ma + em) / 3.0

# Segmentasi
clean_segments = segment_signal(clean_signals, window_size)
noisy_segments = segment_signal(combined_noise, window_size)

print("Shape of clean segments:", clean_segments.shape)
print("Shape of noisy segments:", noisy_segments.shape)

Shape of clean segments: (2539, 256, 1)
Shape of noisy segments: (2539, 256, 1)


## Model Autoencoder

In [48]:
def build_autoencoder(input_shape):
    input_layer = Input(shape=input_shape)

    # ENCODER
    # Blok 1
    x = Conv1D(filters=32, kernel_size=7, padding="same")(input_layer)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = Dropout(0.2)(x)

    # Blok 2
    x = Conv1D(filters=64, kernel_size=7, padding="same")(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = Dropout(0.2)(x)

    # DECODER
    # Blok 3
    x = Conv1D(filters=32, kernel_size=7, padding="same")(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = Dropout(0.2)(x)

    # Layer Output
    # Merekonstruksi sinyal kembali ke 1 channel
    decoded = Conv1D(filters=1, kernel_size=7, activation="tanh", padding="same")(x)

    # Buat dan kompilasi model
    model = Model(input_layer, decoded)
    optimizer = Adam(learning_rate=0.0005)
    model.compile(optimizer=optimizer, loss='mean_squared_error')

    return model

autoencoder = build_autoencoder(input_shape=(window_size, 1))
autoencoder.summary()

Model: "functional_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_12 (InputLayer)     │ (None, 256, 1)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_56 (Conv1D)              │ (None, 256, 32)        │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_44          │ (None, 256, 32)        │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_44 (LeakyReLU)      │ (None, 256, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 256, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_57 (Conv1D)              │ (None, 256, 64)        │        14,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_45          │ (None, 256, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_45 (LeakyReLU)      │ (None, 256, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 256, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_58 (Conv1D)              │ (None, 256, 32)        │        14,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_46          │ (None, 256, 32)        │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_46 (LeakyReLU)      │ (None, 256, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 256, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_59 (Conv1D)              │ (None, 256, 1)         │           225 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 29,761 (116.25 KB)

 Trainable params: 29,505 (115.25 KB)

 Non-trainable params: 256 (1.00 KB)

In [49]:
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

history = autoencoder.fit(
    noisy_segments,
    clean_segments,
    epochs=1000,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping]
)

Epoch 1/1000
64/64 ━━━━━━━━━━━━━━━━━━━━ 9s 56ms/step - loss: 0.2048 - val_loss: 0.0246
Epoch 2/1000
64/64 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 0.1111 - val_loss: 0.0243
Epoch 3/1000
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0798 - val_loss: 0.0263
Epoch 4/1000
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0603 - val_loss: 0.0247
Epoch 5/1000
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0468 - val_loss: 0.0267
Epoch 6/1000
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0382 - val_loss: 0.0269
Epoch 7/1000
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0339 - val_loss: 0.0267
Epoch 8/1000
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0306 - val_loss: 0.0266
Epoch 9/1000
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0284 - val_loss: 0.0256
Epoch 10/1000
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0266 - val_loss: 0.0270
Epoch 11/1000
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0257 - val_loss: 0.0262
Epoch 12/1000
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/ste

## Denoising dan Evaluasi

In [50]:
noisy_datasets = {
    'Baseline Wander': bw,
    'Muscle Artifact': ma,
    'Electrode Motion': em
}

def calculate_rmse(clean, denoised):
    return np.sqrt(np.mean((denoised - clean) ** 2))
def calculate_prd(clean, denoised):
    return 100 * np.sqrt(np.sum((denoised - clean) ** 2) / np.sum(clean ** 2))

for noise_type, signal in noisy_datasets.items():
    noisy_eval_segments = segment_signal(signal, window_size)
    clean_eval_segments = segment_signal(clean_signals, window_size)

    denoised = autoencoder.predict(noisy_eval_segments).reshape(-1, window_size)
    original = clean_eval_segments.reshape(-1, window_size)

    rmses = [calculate_rmse(o, d) for o, d in zip(original, denoised)]
    prds = [calculate_prd(o, d) for o, d in zip(original, denoised)]

    print(f"\nHasil untuk: {noise_type}")
    print(f"Rata-rata RMSE: {np.mean(rmses):.4f}")
    print(f"Rata-rata PRD : {np.mean(prds):.2f}%")


80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step

Hasil untuk: Baseline Wander
Rata-rata RMSE: 0.1447
Rata-rata PRD : 64.19%
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

Hasil untuk: Muscle Artifact
Rata-rata RMSE: 0.1443
Rata-rata PRD : 64.12%
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

Hasil untuk: Electrode Motion
Rata-rata RMSE: 0.1454
Rata-rata PRD : 64.39%
